In [1]:
import cv2
import time
import os
import mediapipe as mp
import HandTrackingModule as htm
from math import atan2

In [2]:
def waveLeft(lmList1, lmList2):
    # The points closest to the left of the window have lower values.
    if (len(lmList1) != 0) and (len(lmList2) != 0):
        # distances between 1st index knuckle and palm of the hand
        # for state 1
        horizontalDistance1 = (lmList1[5][1] - lmList1[0][1])
        verticalDistance1 = (lmList1[9][2] - lmList1[0][2])
        # the angle of the knuckle in state 1
        angleOfKnuckle1 = atan2(verticalDistance1, horizontalDistance1)

        # for state 2
        horizontalDistance2 = (lmList2[5][1] - lmList2[0][1])
        verticalDistance2 = (lmList2[9][2] - lmList2[0][2])
        # the angle of the knuckle in state 1
        angleOfKnuckle2 = atan2(verticalDistance2, horizontalDistance2)
    
        # amount the hand moved between states
        horizontalMovement = lmList2[5][1] - lmList1[5][1]

        # rotating around palm and left horizontal distance
        left = (angleOfKnuckle2 < angleOfKnuckle1) and (horizontalMovement <= -WAVE_HORIZONTAL_DISTANCE)
        
        if left:
            return 1
        else:
            return 0

In [3]:
def waveRight(lmList1, lmList2):
    # The points closest to the left of the window have lower values.
    if (len(lmList1) != 0) and (len(lmList2) != 0):
        # distances between 1st index knuckle and palm of the hand
        # for state 1
        horizontalDistance1 = (lmList1[5][1] - lmList1[0][1])
        verticalDistance1 = (lmList1[9][2] - lmList1[0][2])
        # the angle of the knuckle in state 1
        angleOfKnuckle1 = atan2(verticalDistance1, horizontalDistance1)

        # for state 2
        horizontalDistance2 = (lmList2[5][1] - lmList2[0][1])
        verticalDistance2 = (lmList2[9][2] - lmList2[0][2])
        # the angle of the knuckle in state 1
        angleOfKnuckle2 = atan2(verticalDistance2, horizontalDistance2)

        # amount the hand moved between states
        horizontalMovement = lmList2[5][1] - lmList1[5][1]

        # rotating around palm and right horizontal distance
        right = (angleOfKnuckle2 > angleOfKnuckle1) and (horizontalMovement >= WAVE_HORIZONTAL_DISTANCE)

        if right:
            return 1
        else:
            return 0

In [4]:
wCam, hCam = 640, 480

camera = cv2.VideoCapture(0)

camera.set(3, wCam)
camera.set(4, hCam)

# For importing pictures
#folderPath = "FingerImages"
#myList = os.listdir(folderPath)

#overlayList = []

#for imPath in myList:
    #image = cv2.imread(f'{folderPath}/{imPath}')
    #overlayList.append(image)

pTime = 0

MAX_N_FRAMES = 5
# horizontal distance of 2 right / frame seems reasonable
WAVE_HORIZONTAL_DISTANCE_PER_FRAME = 2
WAVE_HORIZONTAL_DISTANCE = (MAX_N_FRAMES * WAVE_HORIZONTAL_DISTANCE_PER_FRAME)

detector = htm.handDetector(detectionCon = 0.75)

In [5]:
# current state of the landmark list (initialize to empty list)
lmList = []
prevLmList = []
frameCount = 0

while True:
    success, frame = camera.read()
    
    frame = cv2.flip(frame, 1)
    
    frame = detector.findHands(frame)
    
    # update both lists
    if (frameCount < MAX_N_FRAMES):
        frameCount += 1
    else:
        prevLmList = lmList
        lmList = detector.findPosition(frame, draw = False)
        frameCount = 0
    
    if waveLeft(prevLmList, lmList):
        cv2.putText(frame, f'Waving Left', (0, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 3)
    elif waveRight(prevLmList, lmList):
        cv2.putText(frame, f'Waving Right', (0, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 3)
   
    
    #Code for images
    #h, w, c = overlayList[0].shape
    #img[0:h, 0:w] = overlayList[0]
    
    
    # Calculating the Frames per Seconds (FPS)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    
    # Displaying the FPS (ERROR PRINTING HIGH NUMBERS)
    cv2.putText(frame, f'FPS: {int(fps)}', (400, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0,0,0), 3)
    
    cv2.imshow("Camera", frame)
    
    key = cv2.waitKey(1)
    if key in [27, ord('Q'), ord('q')]:
        break
        
camera.release()
cv2.destroyAllWindows()